In [1]:
#query2 DATAFRAME with .csv for comparison with .parquet

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1266,application_1732639283265_1229,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import substring
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import count, when, length
import time

from pyspark.sql.functions import count, when, col
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, round

spark = SparkSession \
    .builder \
    .appName("DF query 2 execution") \
    .getOrCreate()

# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3008,application_1732639283265_2966,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [2]:
start_time = time.time()

joined_data = spark.read.csv(
    "s3://groups-bucket-dblab-905418150721/group17/query2_data.csv",
    header=True
)

#joined_data.show(20)
#we read the .csv with all the data in order to compare it to .parquet

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#joined_data.show(3)
result_total_cases = (
    joined_data.groupBy("DIVISION", "YEAR")  # Group by 'year' and 'area'
    .agg(count(when(col("Status") != "VAGGELIS", 1)).alias("Total cases"))).orderBy((["YEAR", "Total Cases"]), desceding=[True,True])

# Show the result
# result_total_cases.show(22)


result_open_cases = (
    joined_data.groupBy("DIVISION", "YEAR")
    .agg(
        count(
            when(
                (col("Status") == "UNK") | (col("Status") == "CC") | col("Status").isNull() | (col("Status") == "IC"), 1
            )
        ).alias("Open Cases") 
    )
    .orderBy(col("Open Cases").desc())
)

# Show the result
# result_open_cases.show(22)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# joined_data = joined_data.join(police_station, joined_data['AREA '] == police_station['PREC'], 'inner')
# df1.join(df2, (df1["column1"] == df2["column1"]) & (df1["column2"] == df2["column2"]), "join_type")


final_data = result_total_cases.join(
    result_open_cases, 
    (result_total_cases["YEAR"] == result_open_cases["YEAR"]) & (result_total_cases["DIVISION"] == result_open_cases["DIVISION"]),
    "inner"
)
final_data = final_data.drop(result_open_cases["DIVISION"]).drop(result_open_cases["YEAR"])

final_data = final_data.withColumn(
    "closed_case_rate", 
    100*(1 - (col("Open Cases") / col("Total Cases")))
)
final_data = final_data.withColumnRenamed("DIVISION", "precinct") \
                       .withColumnRenamed("YEAR", "year")
final_data = final_data.drop("Open Cases", "Total Cases")



window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

final_data = final_data.withColumn("ranking", row_number().over(window_spec))

final_data = final_data.orderBy("year", "ranking")

final_data.select("year", "precinct", "closed_case_rate", "ranking")

final_data.show()

end_time = time.time()
elapsed_time = end_time - start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----+------------------+-------+
|        precinct|year|  closed_case_rate|ranking|
+----------------+----+------------------+-------+
|         RAMPART|2010| 32.84713448949121|      1|
|         OLYMPIC|2010|31.515289821999083|      2|
|          HARBOR|2010| 29.36028339237341|      3|
|     WEST VALLEY|2010| 25.97907001834071|      4|
|      HOLLENBECK|2010|25.209289765055367|      5|
|       NORTHEAST|2010|24.705993037915142|      6|
|       SOUTHEAST|2010|24.034296028880863|      7|
|          NEWTON|2010|23.422342234223425|      8|
|        VAN NUYS|2010|21.825750817722266|      9|
| NORTH HOLLYWOOD|2010|20.818301803783545|     10|
|       SOUTHWEST|2010| 18.88270610631132|     11|
|       HOLLYWOOD|2010|16.948968512486427|     12|
|         CENTRAL|2010| 16.94654352085082|     13|
|        WILSHIRE|2010|15.355484974346446|     14|
|         PACIFIC|2010| 9.528531506355364|     15|
|         TOPANGA|2010| 6.825284806936183|     16|
|        FOOTHILL|2010| 5.63636

In [5]:
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 16.51 seconds